In [7]:
import json
import os
import time
from datetime import date, datetime, timedelta
from pathlib import Path

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [8]:
def get_col():
    return [
        'query_date',
        'flight_no',
        'flight_type',
        'departure_airport',
        'departure_airport_code_1',
        'departure_airport_code_2',
        'arrival_airport',
        'arrival_airport_code_1',
        'arrival_airport_code_2',
        'link',
        'sync'
    ]


def get_soup(corp, start_page, ss):
    url = f'https://www.flightaware.com/live/fleet/{corp}?;offset={start_page};order=ident;sort=ASC'
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36'}
    res = ss.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    return soup


def page_exist_or_not(soup):
    page_exist = True
    for tag in soup.find_all('i'):
        if tag.text == "Sorry. No matching flights found; try again later.":
            page_exist = False

    return page_exist


def split_airport_code(code):
    """若機場代碼有兩種形式，會將兩者分開，回傳兩個代碼"""
    if code is not None:
        code = code.replace("(", "").replace(")", "")
        if "/" in code:
            code1, code2 = code.split("/")
            code1 = code1.strip()
            code2 = code2.strip()
        else:
            code1 = code
            code2 = code
    else:
        code1 = None
        code2 = None

    return code1, code2


def safe_extract(func):
    """判斷一個soup物件是否存在/有值，若沒有則回傳None"""
    try:
        return func()
    except (IndexError, AttributeError):
        return None


def get_flight_info(table_list, logtime):
    page_data = []
    for i in table_list:
        single = []
        flight_no = i('td')[0].span.a.text
        print(f'查詢{flight_no}班機資料...')

        # 紀錄日期
        single.append(logtime)

        # 班機編號
        single.append(safe_extract(lambda: i('td')[0].span.a.text))

        # 機型
        single.append(safe_extract(lambda: i('td')[1].span.a.text))

        # 起飛機場
        single.append(safe_extract(lambda: i('td')[2]('span', dir='ltr')[0].text))

        # 起飛機場代號（如有兩種則分開儲存，只有一種則重複儲存）
        code_d = safe_extract(lambda: i('td')[2]('span', dir='ltr')[1].text)
        code_d1, code_d2 = split_airport_code(code_d)
        single.append(code_d1)
        single.append(code_d2)

        # 降落機場
        single.append(safe_extract(lambda: i('td')[3]('span', dir='ltr')[0].text))

        # 降落機場代號（如有兩種則分開儲存，只有一種則重複儲存）
        code_a = safe_extract(lambda: i('td')[3]('span', dir='ltr')[1].text)
        code_a1, code_a2 = split_airport_code(code_a)
        single.append(code_a1)
        single.append(code_a2)

        # 連結
        single.append('https://www.flightaware.com' + safe_extract(lambda: i('td')[0].span.a['href']))

        # 同步標記
        single.append(0)

        # 存回page_data list
        page_data.append(single)

    return page_data

In [ ]:
flight_corp = ["EVA", "CAL", "SJX", "TTW"]

for corp in flight_corp:
    # 判斷總列表.csv檔是否存在，若不存在則先建立一個只有columns的空表格
    folder = r"C:\Users\add41\Documents\Data_Engineer\Project\Flights-Data-Crawler\Data"
    file = f"{corp}_FlightList.csv"
    file_path = os.path.join(folder, file)
    path = Path(file_path)

    if path.exists():
        df_main = pd.read_csv(file_path)
    else:
        columns = get_col()
        df_main = pd.DataFrame(columns=columns)


    # 建立空list（為建立dataframe預備）並設定起始頁數，建立ss連線
    data = []
    logtime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    start_page = 0
    ss = requests.Session()

    while True:
        # 網頁為20筆一頁，設定從第0筆開始查詢，每次回圈+20，直到查無資料後break
        # 利用ss.get發出請求並轉換出soup物件
        soup = get_soup(corp, start_page, ss)
        print(f'開始查詢{corp}的第{start_page}到{start_page + 20}筆資料...')

        # 若仍有資料 found = False 則繼續迴圈，分別尋找兩個標籤（兩種皆有連結）並合併list
        if page_exist_or_not(soup):
            table_list = soup('table', class_='prettyTable fullWidth')[0]('tr')[2:]
            page_data = get_flight_info(table_list, logtime)

            for single_data in page_data:
                data.append(single_data)

            # 完成後查詢筆數+20並稍微等待後在進行下一次迴圈
            print(f'完成存取{corp}的第{start_page}到{start_page + 20}筆資料')
            start_page += 20
            time.sleep(10)

        # 當查無資料時 found = True 顯示查無資料並終止迴圈
        else:
            print(f'{corp}沒有第{start_page}到{start_page + 20}筆資料')
            break

    print(f'已完成{corp}存取資料')


    # 根據爬蟲資料建立Dataframe
    columns = get_col()

    df_corp = pd.DataFrame(columns=columns, data=data)
    print(f'{corp}新資料建檔完成')

    # 直接將新資料與舊資料合併
    df_main = pd.concat([df_main, df_corp], ignore_index=True)

    # 對合併後的資料使用drop_duplicates，將重複值刪去，並覆蓋回df_main
    df_main = df_main.drop_duplicates(subset='link', keep = 'first').reset_index(drop=True)

    # 將query_date欄位轉換為datetime物件
    df_main['query_date'] = pd.to_datetime(df_main['query_date'])

    # 將新的df_main進行存檔
    df_main.to_csv(file_path, index=False)

    print(f'完成{corp}資料更新，目前資料筆數：{len(df_main)}')
    print('5秒後繼續...')
    time.sleep(5)

print('已完成所有航空公司資料更新！')